In [2]:
import pandas as pd

In [3]:
df = pd.read_excel(r'C:\Users\ilyx\Documents\GitHub\REVIEW\xlsx\review_preprocessing.xlsx')

In [4]:
lens = [] 
for i in df.Rewiew:
    lens.append(len(str(i)))

In [5]:
df['LenReview'] = lens
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,rewiew_ID,article,Date,Name,Rating,Rewiew,amount Photo,LenReview
0,0,0.0,0.0,y1a2e4cBpT2rITcGXb5U,123837570.0,13-04-2023 17:42:42,Владимир,3,начинать подтупливать сенсор становиться работ...,0,57
1,1,18.0,18.0,-DXpUIcBxpMXtsmOPat0,123837570.0,05-04-2023 10:14:36,Евгений,3,телефон быстрый хороший оболочка глючать хлипк...,0,364
2,2,31.0,31.0,ojLKMYcBMBFuPsko1ha6,123837570.0,30-03-2023 09:13:10,Наталья,3,телефон купить декабрь спустя месяц начинать в...,0,139
3,3,39.0,39.0,jl4JKIcBtPujtG5sD7mq,123837570.0,28-03-2023 11:44:55,Артем,3,телефон неплохой недочет приходить скрепка защ...,0,145
4,4,42.0,42.0,0q9bH4cBJ99b87NGoOGL,123837570.0,26-03-2023 19:18:31,Александр,4,телефон плохой почемуто обновление система ста...,0,111
...,...,...,...,...,...,...,...,...,...,...,...
7428,7428,NaN,NaN,NaN,NaN,NaN,NaN,3,товар отличный,0,14
7429,7429,NaN,NaN,NaN,NaN,NaN,NaN,3,разбираться никак обои поменять вопрос часы от...,0,52
7430,7430,NaN,NaN,NaN,NaN,NaN,NaN,3,отказ зарядка цвет приходить,0,28
7431,7431,NaN,NaN,NaN,NaN,NaN,NaN,3,сутки 2 держать заряд :sparkling_heart: понрав...,0,51


In [6]:
df['isReal'] = lens

In [7]:
for i in range(len(df)):
    if df['LenReview'].loc[i] < 67 and df['Rating'].loc[i] < 3:
        df['isReal'].loc[i] = 0
    else:
        df['isReal'].loc[i] = 1

C:\Users\ilyx\AppData\Local\Temp\ipykernel_4440\3454778796.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isReal'].loc[i] = 1
C:\Users\ilyx\AppData\Local\Temp\ipykernel_4440\3454778796.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isReal'].loc[i] = 0


In [8]:
df['text'] = df.apply(lambda row: ' '.join([str(row['Rewiew']),'Rating:', str(row['Rating']), 'Amount Photo:', str(row['amount Photo']), 'LenReview:', str(row['LenReview'])]), axis=1)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df.isReal

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
gb_clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=3, random_state=42)

# Обучение модели на обучающей выборке
gb_clf.fit(X_train, y_train)

# Получение предсказаний на тестовой выборке
y_pred = gb_clf.predict(X_test)

# Вычисление точности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8722259583053127


In [12]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(X_train, y_train)

c:\Users\ilyx\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
# Получение предсказаний на тестовой выборке
y_pred = lg.predict(X_test)

# Вычисление точности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8628110289172831


In [14]:
import pickle
with open('isReal_model_log_reg.pickle', 'wb') as f:
    pickle.dump(lg, f)

In [15]:
df.to_excel(r'C:\Users\ilyx\Documents\GitHub\REVIEW\xlsx\review_preprocessing_with_real.xlsx')